In [1]:
import deepxde.deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, Tensor
from torch.nn import functional as F

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [ ]:
def L2_loss(inputs: Tensor, targets: Tensor, aux: np.ndarray, outputs: Tensor, model) -> Tensor | list[Tensor]:
    return F.mse_loss(outputs, targets)

def PINN_loss(inputs: Tensor, targets: Tensor, aux: np.ndarray, outputs: Tensor, model) -> Tensor | list[Tensor]:
    return F.mse_loss(outputs, targets)

In [ ]:
import deepxde.deepxde as dde
import matplotlib.pyplot as plt
import numpy as np


# PDE
def pde(x, y, v):
    D = 0.01
    k = 0.01
    print("0", end=" ")
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, j=0)
    return dy_t - D * dy_xx + k * y**2 - v


geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc = dde.icbc.DirichletBC(geomtime, lambda _: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda _: 0, lambda _, on_initial: on_initial)

pde = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=200,
    num_boundary=40,
    num_initial=20,
    num_test=100,
)

# Function space
func_space = dde.data.GRF(length_scale=0.2)

# Data
eval_pts = np.linspace(0, 1, num=50)[:, None]
data = dde.data.PDEOperatorCartesianProd(
    pde, func_space, eval_pts, 10, function_variables=[0], num_test=10
)

# Net
net = dde.nn.DeepONetCartesianProd(
    [50, 128, 128, 128],
    [2, 128, 128, 128],
    "tanh",
    "Glorot normal",
)

model = dde.Model(data, net)
model.compile("adam", lr=0.0005)
losshistory, train_state = model.train(epochs=50000, batch_size = None)
dde.utils.plot_loss_history(losshistory)

Compiling model...
'compile' took 0.000279 s

Training model...

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Step      Train loss                        Test loss                         Test metric
0         [1.05e+00, 8.74e-02, 3.94e-02]    [1.62e+00, 1.01e-01, 4.97e-02]    []  
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: 

In [ ]:
# PDE
def pde(x, y, v):
    D = 0.01
    k = 0.01
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, j=0)
    return dy_t - D * dy_xx + k * y**2 - v


geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc = dde.icbc.DirichletBC(geomtime, lambda _: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda _: 0, lambda _, on_initial: on_initial)

pde = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=200,
    num_boundary=40,
    num_initial=20,
    num_test=500,
)

# Function space
func_space = dde.data.GRF(length_scale=0.1)

# Data
eval_pts = np.linspace(0, 1, 101)[:, None]
data = dde.data.PDEOperator(pde, func_space, eval_pts, 1000, function_variables=[0], num_test=1000)

# Net
net = dde.nn.DeepONet(
    [101, 128, 128, 128],
    [2, 128, 128, 128],
    "tanh",
    "Glorot normal",
)

model = dde.Model(data, net)
model.compile("adam", lr=0.0005)
losshistory, train_state = model.train(epochs=50000)
dde.utils.plot_loss_history(losshistory)

func_feats = func_space.random(1)
xs = np.linspace(0, 1, num=100)[:, None]
v = func_space.eval_batch(func_feats, xs)[0]
x, t, u_true = solve_ADR(
    0,
    1,
    0,
    1,
    lambda x: 0.01 * np.ones_like(x),
    lambda x: np.zeros_like(x),
    lambda u: 0.01 * u**2,
    lambda u: 0.02 * u,
    lambda x, t: np.tile(v[:, None], (1, len(t))),
    lambda x: np.zeros_like(x),
    100,
    100,
)
u_true = u_true.T
plt.figure()
plt.imshow(u_true)
plt.colorbar()

v_branch = func_space.eval_batch(func_feats, np.linspace(0, 1, num=50)[:, None])[0]
xv, tv = np.meshgrid(x, t)
x_trunk = np.vstack((np.ravel(xv), np.ravel(tv))).T
u_pred = model.predict((np.tile(v_branch, (100 * 100, 1)), x_trunk))
u_pred = u_pred.reshape((100, 100))
print(dde.metrics.l2_relative_error(u_true, u_pred))
plt.figure()
plt.imshow(u_pred)
plt.colorbar()
plt.show()

In [ ]:
import inspect
from typing import Callable, Any
def is_pinn(func: Callable[[Any], Any]) -> bool:
    func_sig = inspect.signature(func).parameters
    if all(i in func_sig for i in ["inputs", "outputs"]):
        return True
    else:
        return False

def is_data(func: Callable[[Any], Any]) -> bool:
    func_sig = inspect.signature(func).parameters
    if all(i in func_sig for i in ["outputs", "targets"]):
        return True
    else:
        return False

def is_mix(func: Callable[[Any], Any]) -> bool:
    if is_pinn(func) and is_data(func):
        return True
    else:
        return False

In [ ]:
def a(inputs, aux, targets, model):
    return 1

is_pinn(a)

False